# NOTE
This notebook was created for the sole purpose of generating maps for the project website; the code was run locally and might be slightly deprecated due to small changes between the datasets used here and the finalized versions linked to from Google Drive

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool

In [2]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
#Download from: https://drive.google.com/open?id=1AOv3CrNOysm8FOfgSLL8aWhcWGPRXgPb
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [4]:
#Download from: https://drive.google.com/open?id=1-AtGVar_xNunGONvi1ANozOna3ZTriU8
listings = pd.read_csv('/home/novais/Downloads/listings.csv')

In [5]:
listings.head()

,Unnamed: 0,listing_id,host_id,neighbourhood,latitude,longitude,room_type,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,lr_m,lr_d,lr_y,lat,lon,location,loc,loc_id
0,0,2384,2613,Hyde Park,41.78886,-87.58671,2,159,2.89,1,306,7,11,2019,41.79,-87.59,"(41.78886, -87.58671)","(41.79, -87.59)",124
1,1,4505,5775,South Lawndale,41.85495,-87.69696,1,373,2.96,1,326,6,25,2019,41.85,-87.70,"(41.85495, -87.69696)","(41.85, -87.7)",190
2,2,7126,17928,West Town,41.90289,-87.68182,1,349,2.86,1,323,6,30,2019,41.90,-87.68,"(41.90289, -87.68182)","(41.9, -87.68)",269
3,3,9811,33004,Lincoln Park,41.91769,-87.63788,1,38,0.59,8,327,6,30,2019,41.92,-87.64,"(41.91769, -87.63788000000001)","(41.92, -87.64)",310
4,4,10610,2140,Hyde Park,41.79708,-87.59195,2,39,0.68,4,247,6,6,2019,41.80,-87.59,"(41.79708, -87.59195)","(41.8, -87.59)",140


In [6]:
listings_locations = listings[['loc','loc_id']]
listings_locations = listings_locations.drop_duplicates('loc_id')
listings_locations = listings_locations.set_index('loc_id')
listings_locations = listings_locations.sort_index()
listings_locations = listings_locations.reset_index()
listings_locations['loc_id2']=listings_locations.index
list_locs=list(listings_locations['loc'])
len(list_locs)

370

In [7]:
listings=listings.merge(listings_locations,on=['loc_id','loc'])
type(listings.lat[0])

numpy.float64

In [8]:
airbnb_cr.head()

,Unnamed: 0,listing_id,date,price,month,day,year,host_id,neighbourhood,room_type,...,date_id,crimes_loc_1km,crimes_loc_2km,crimes_loc_5km,crimes_1km,crimes_2km,crimes_5km,crimes_1km_count,crimes_2km_count,crimes_5km_count
0,0,2384,2018-07-26,75.0,7,26,2018,2613,Hyde Park,2,...,17,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...","[244, 246]","[224, 225, 264, 265, 266, 633, 223]","[262, 263, 279, 281, 162, 163, 164, 165, 166, ...",4,14,89
1,268,2384,2018-07-29,69.0,7,29,2018,2613,Hyde Park,2,...,20,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...",[],"[224, 225, 265, 266, 267, 243, 223]","[262, 263, 278, 279, 280, 281, 289, 162, 163, ...",0,20,76
2,847,2384,2018-08-05,65.0,8,5,2018,2613,Hyde Park,2,...,27,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...",[246],"[265, 266, 267, 243, 223]","[262, 263, 280, 281, 289, 162, 163, 164, 165, ...",1,12,94
3,1343,2384,2018-08-12,75.0,8,12,2018,2613,Hyde Park,2,...,34,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...","[244, 246]","[224, 225, 265, 266, 267, 223]","[263, 278, 279, 289, 162, 163, 164, 165, 166, ...",3,14,84
4,1976,2384,2018-10-01,65.0,10,1,2018,2613,Hyde Park,2,...,84,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...",[244],"[224, 264, 265, 266, 223]","[263, 279, 280, 281, 289, 162, 163, 164, 165, ...",1,7,86


In [9]:
ids=list(airbnb_cr['listing_id'].unique())

In [10]:
listings=listings[listings.listing_id.isin(ids)]

In [11]:
loc_count=airbnb_cr.groupby(['listing_id']).mean()
loc_count=loc_count['loc_id'].astype(int)
loc_count=loc_count.reset_index()
loc_count=loc_count.groupby(['loc_id']).count()
loc_count=loc_count.reset_index()
loc_count=loc_count.rename(index=str, columns={"loc_id": "loc_id2","listing_id": "listings_count"})
loc_count.head()

,loc_id2,listings_count
0,0,1
1,1,1
2,2,1
3,3,2
4,4,1


In [12]:
listings2=listings.merge(loc_count,on='loc_id2')
listings2=listings2.groupby('loc_id2').mean()
listings2=listings2.reset_index()
listings2=listings2[['lat','lon','loc_id2','listings_count']]
listings2['loc']=list(zip(listings2.lat,listings2.lon))
listings2.head()

,lat,lon,loc_id2,listings_count,loc
0,41.65,-87.54,0,1.0,"(41.65, -87.54)"
1,41.66,-87.55,1,1.0,"(41.66, -87.55)"
2,41.67,-87.66,2,1.0,"(41.67, -87.66)"
3,41.69,-87.68,3,2.0,"(41.69, -87.68)"
4,41.69,-87.67,4,1.0,"(41.69, -87.67)"


In [13]:
listings2=listings2.drop_duplicates(subset='loc')

In [14]:
listings2.head()

,lat,lon,loc_id2,listings_count,loc
0,41.65,-87.54,0,1.0,"(41.65, -87.54)"
1,41.66,-87.55,1,1.0,"(41.66, -87.55)"
2,41.67,-87.66,2,1.0,"(41.67, -87.66)"
3,41.69,-87.68,3,2.0,"(41.69, -87.68)"
4,41.69,-87.67,4,1.0,"(41.69, -87.67)"


In [15]:
listings2.listings_count.describe()

count    341.000000
mean      12.516129
std       17.011169
min        1.000000
25%        2.000000
50%        5.000000
75%       16.000000
max       93.000000
Name: listings_count, dtype: float64

In [16]:
type(listings2['loc'][0])

tuple

In [17]:
# listings2['loc'] = listings2['loc'].apply(lambda x: literal_eval(x))
listings2['coords_x'] = listings2['loc'].apply(lambda x: merc(x)[0])
listings2['coords_y'] = listings2['loc'].apply(lambda x: merc(x)[1])

p1 = figure(title = "Listings Count")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

l1 = listings2[listings2['listings_count']<=2]
l2 = listings2[(listings2['listings_count']>2)&(listings2['listings_count']<=5)]
l3 = listings2[(listings2['listings_count']>5)&(listings2['listings_count']<=15)]
l4 = listings2[(listings2['listings_count']>15)&(listings2['listings_count']<=50)]
l5 = listings2[listings2['listings_count']>50]

source1 = ColumnDataSource(data=dict(
    count=l1['listings_count'],
    cx=l1['coords_x'],
    cy=l1['coords_y'],
    sizes = [10]*len(l1),
    names = ['< 2']*len(l1),
    colors = ['#79D151']*len(l1)
    ))
source2 = ColumnDataSource(data=dict(
    count=l2['listings_count'],
    cx=l2['coords_x'],
    cy=l2['coords_y'],
    sizes = [10]*len(l2),
    names = ['[2,5]']*len(l2),
    colors = ['#22A784']*len(l2)
    ))
source3 = ColumnDataSource(data=dict(
    count=l3['listings_count'],
    cx=l3['coords_x'],
    cy=l3['coords_y'],
    sizes = [10]*len(l3),
    names = ['(5,15]']*len(l3),
    colors = ['#29788E']*len(l3)
    ))
source4 = ColumnDataSource(data=dict(
    count=l4['listings_count'],
    cx=l4['coords_x'],
    cy=l4['coords_y'],
    sizes = [10]*len(l4),
    names = ['(15,50]']*len(l4),
    colors = ['#404387']*len(l4)
    ))
source5 = ColumnDataSource(data=dict(
    count=l5['listings_count'],
    cx=l5['coords_x'],
    cy=l5['coords_y'],
    sizes = [10]*len(l5),
    names = ['> 50']*len(l5),
    colors = ['#440154']*len(l5)
    ))

p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',source=source1,size='sizes',color = 'colors',legend='names')
p1.circle(x = 'cx',y = 'cy',source=source2,size='sizes',color = 'colors',legend='names')
p1.circle(x = 'cx',y = 'cy',source=source3,size='sizes',color = 'colors',legend='names')
p1.circle(x = 'cx',y = 'cy',source=source4,size='sizes',color = 'colors',legend='names')
p1.circle(x = 'cx',y = 'cy',source=source5,size='sizes',color = 'colors',legend='names')

p1.add_tools(HoverTool(
    tooltips=[
        ( 'Nº of listings', '@count')
    ],
    
    formatters={
        'Nº of listings' : 'numeral',   
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [18]:
output_notebook()


Loading BokehJS ...

In [20]:
show(p1)

In [21]:
print(p1_script)


<script type="text/javascript">
  (function() {
    var fn = function() {
      Bokeh.safely(function() {
        (function(root) {
          function embed_document(root) {
            
          var docs_json = '{"3a3f6492-0dc8-49c5-b4e4-f9f6e5b9774e":{"roots":{"references":[{"attributes":{"fill_color":{"field":"colors"},"line_color":{"field":"colors"},"size":{"field":"sizes","units":"screen"},"x":{"field":"cx"},"y":{"field":"cy"}},"id":"1103","type":"Circle"},{"attributes":{"fill_alpha":{"value":0.1},"fill_color":{"value":"#1f77b4"},"line_alpha":{"value":0.1},"line_color":{"value":"#1f77b4"},"size":{"field":"sizes","units":"screen"},"x":{"field":"cx"},"y":{"field":"cy"}},"id":"1104","type":"Circle"},{"attributes":{"callback":null,"data":{"colors":["#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E","#29788E

In [22]:
print(p1_div)


<div class="bk-root" id="f403471f-d562-4dbe-b1a7-c5394ba46c65"></div>
